# Using Julia to do modern Reinforcement Learning Research

Looking at the Julia's current surge in popularity in scientific fields and the current amazing work being done in the open source community towards using RL, it makes sense to start considering Julia for doing researching in the machine learning fields. 

This series of tutorials will act as an introduction to using julia to do research in reinforcement learning. We will talk about important details when constructing experiments (for reproducibility) and several facets of Julia which make life much easier for researchers. These tutorials were constructed with jupyter notebooks.

# Why Julia


You may be asking yourself, why should we think about using Julia as Python is ubiquious in the field? This is a great question, and one many people currently using Julia have also thought through. I won't try and convince you here, but here are a few reasons I have for using Julia:

* Mulitple dispatch makes code simpler as compared to Object Orientated Programming.
* Linear algebra accelerated through BLAS is built in to the language. (i.e. arrays of numbers are BLAS by default).
* Plotting code is usually simpler using Plots.jl.

These are only a few reasons, but those I find most beautiful about julia. As we develop our reinforcement learning framework from scratch, I would like you to consider how you would implement these ideas in python. For example, what libraries would you need? What class structure? How would I change behavior? I've found my code significantly simplifies as I try to do things the Julia way (taking full advantage of multiple dispatch).




## Linear Algebra

Lets start simple, and perform some typical linear algebra operations using Julia to get a handle of the language. While these contain only a subset of what is baked into julia, you should be able to extrapolate to other operations you care about. 



In [ ]:
a = rand(2) # get a random column vector of size 10 and set it to the variable a
b = rand(2)
s = rand()
M = rand(2,2)

println("vector addition")
@show a + b

println("Scalar multiplication:")
@show 2a
@show 2*a

println("Element wise vector multiplication:")
@show a .* b

println("Inner product of vectors")
@show a' * b

println("Outer product of vectors")
@show a * b'

println("Matrix mulitplication of vectors")
@show M * a

println("Matrix element wise product with outer product operation")
@show M .* (a * b')

println("Broadcast scalar-vector addtion")
@show a .+ s

; # ignore this semi-colon.

Along with these operations which are always available, there is a Base package `L


If you have a specfic linear algebra operation you want but can't find it in Base, you will need to explicitly load the LinearAlgebra package. This is already available to you in the STDLib 


In [ ]:
using LinearAlgebra

println("Singular Value Decomposition")
M_svd = svd(M)
@show M_svd.U
@show M_svd.S
@show M_svd.V

# Check that M = USV' to some floating point error (i.e. approximately)
all((M_svd.U * diagm(M_svd.S) * M_svd.Vt) .≈ M)


# Random Numbers

Being considerate about your random number generators is one of the most important aspects of making experiments reproducible (i.e. setting your random seed). Julia lets you set the seed of a Global random number generator, as well as construct and manage your own.


In [ ]:
using Random # Already in the language, you are just accessing the namespace

Random.seed!(Random.GLOBAL_RNG, 10) # Set global random seed

@show all(rand(2) .== [0.11258244478647295, 0.36831406658084287])

rng = Random.MersenneTwister(10)

@show all(rand(rng, 2) .== [0.11258244478647295, 0.36831406658084287])


;

# Multiple Dispatch

Multiple dispatch is the central design ideology of Julia (much like OOP is central to Python or Java). At first glance, it seems very similar to function overloading of other languages (i.e. C++), but it has much more utility because of the ability to dispatch on all argument types (not just one or two)!  This will be useful later, for now I am only going to simple show how you can take advantage.


In [ ]:

function f(x)
    return "default"
end

function f(x::Integer)
    return "Int"
end

function f(x::AbstractFloat)
    return "Float"
end


println(f("Hello")) # pass f a string
println(f(1)) # pass f a integer
println(f(200f0)) #pass f a Float32 (single precision floating point number)


A method is Julia's term for a specialized version of a function. Above we wrote a function `f` and hand-made specialized methods for integers and floats. While this may seem like the compiler is only working on the specialized versions, this is incorrect! The compiler will create a specialized method automatically from the generic function, meaning you get the performance of a hand-specialized method. The overriden methods are useful for when there are code changes for different types (which we'll see later on).

If you specialize a function w/o a generic fallback version you will get an exception that there is no matching method.

In [ ]:
function greet(s::String)
    println("Hello $(s)")
end

greet("Matthew")
greet(1) # This should throw an exception! greet is not defined for integers!

Later in the series you will see how to take advantage of multiple dispatch to design an RL interface and use it to make design easier with composition.

## Types and Data

Now that we have some of the fundamental building blocks of what makes julia tick, we can start thinking about custom types. First, lets just build a basic struct which contains some data we can act on. As a simple example, lets make a struct A which stores an integer (you can imagine this struct being an agent, environment, or really anything), with a simple function.

In [ ]:
struct A
   data::Int
end

function double(a::A)
    a.data * 2
end

This just returns double the data stored in A. Lets make another struct B which holds a string this time

In [ ]:
struct B
   data::String
end

we can dispatch on my_func by specializing:

In [ ]:
function double(b::B)
    ret = tryparse(Int, b.data)
    if ret == nothing
        0
    else
        2*ret
    end
end

This parses the data in b as an Int and doubles. If it is unable to parse (i.e. the data isn’t an Int) it returns 0.

Great!

Now we can use this in a more complex, but general function

```julia
function complicated_function(a_or_b, args...)
    # ... Stuff goes here ...
    data_doubled = double(a_or_b)
    # other stuff
end
```

Notice how I didn’t specialize the a_or_b parameter above and instead kept it generic. This means any struct which specializes double will slot in the correct function when complicated_function is compiled!

Now it should be pretty clear how you can use multiple dispatch to get the kind of generics you are wanting (even though these are contrived examples). We can abstract one more layer and make this even more usable using abstracts:

In [ ]:
abstract type AbstractAB end

double(aab::AbstractAB) = data_as_int(aab)*2

my_func(aab::AbstractAB) = 20*(data_as_int(aab)^2 + 10)

struct A
   data::Int
end

data_as_int(a::A) = a.data

struct B
   data_str::String
end

function data_as_int(b::B) 
   ret = tryparse(Int, b.data_str)
   if ret == nothing
      0
   else
      ret
   end
end

Notice that we moved the complex and specialized code into more restrictive functions so the general functions can be reused. While here we used actual Abstract typing to make dispatch work the way we want, you can also build this exact same interface using duck typing!

# Design patterns

While I won't go into detail about design patterns which emerge from Julia's multiple dispatch and typing system, you should read [this blog](https://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/) by Christopher Rackauckas (who is an active user of the language doing research in applying ML/AI research to Scientific pursuits). 
